# Project File - APS360 Team 25
Divided into the following section: 
# 
1) Library imports
2) Data imports
3) Model architecture definition
4) Training function definition
5) Model training
6) Model testing

## Library imports 
(Place all library imports here)

In [1]:
#KP - I just added the main ones from the labs.
import matplotlib.pyplot as plt
import numpy as np

#import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, random_split

import time # Tracking model training time.

# Install mido for Data importing
'''
Pip install for COLAB:
'''
# !pip install mido;
import mido
from mido import MidiFile, Message, MidiTrack, MetaMessage
import os
import random
import copy

In [2]:
#Set working directory if required:
''' For LOCAL:
'''
os.chdir('D:\engsci\year 3\CLASS\APS360\project_pivot') #Sets current working directory!

## Data imports
#### MIDI reading functions

In [3]:
def CountTracks(directory):          #Count files and tracks in folder
    trackCount = 0
    fileCount = 0
    for file in os.listdir(directory):
        if file.endswith(".midi"):
            fileCount += 1
            midiDir = MidiFile(directory+"/"+file)
            for track in midiDir.tracks:
                trackCount += 1
    print(fileCount+" files")
    print(trackCount+" tracks")

    
def PrintMessages(mid):                # print midi messages
    for i, track in enumerate(mid.tracks):
        print('Track {}: {}'.format(i, track.name))
        for msg in track:
            print(msg)

            
def PrintSomeMessages(mid):             #print first 200 midi messages
    track = mid.tracks[1]
    for i,msg in enumerate(track):
        if i < 200:
            print(msg)
            
def PrintMetaMessages(mid):             #print fmeta messages
    track = mid.tracks[0]
    for i,msg in enumerate(track):
        print(msg)

def cleanupMessages(mid):              #removes non-note messages by force
    track = mid.tracks[1]
    track2 = []
    for msg in track:
        if msg.type == "note_on":
            track2.append(msg)
    mid.tracks[1] = track2

#### MIDI to Numpy code

In [4]:
def Midi2NumpyNoSustain(mid, track0=1):                                #converts to numpy array removing non-note messages
    track = mid.tracks[track0]                           #0th track only contains meta-messages, all notes on 1st track
    notes = np.empty([0,4])
    time = 0
    for msg in track:
        if msg.type == "note_on":                   # only count "note" messages - other inputs i.e. foot pedals are ignored
            notes = np.append(notes,np.array([[msg.note, msg.velocity, msg.time + time, 0]]),axis=0)         # (note, velocity, time, sustain)
            time = 0
        elif msg.type == "note_off":
            notes = np.append(notes,np.array([[msg.note, 0, msg.time + time, 0]]),axis=0)         # (note, velocity, time, sustain)
            time = 0        
        else:
            time += msg.time                        #adjust time when removing other messages
    return notes


def NumpyGetSustain(note):
    notes = np.copy(note)
    for i, msg in enumerate(notes):
        if msg[1] > 0:                            # if velocity is not 0
            j = 1
            sustain = 0
            while msg[0] != notes[i+j][0]:        # while note values are different
                sustain += notes[i+j][2]
                j += 1                            #search for next message with same note i.e. message telling that note was released
            notes[i,3] = sustain + notes[i+j][2]
    time = 0
    for i, msg in enumerate(notes):
        if msg[1] > 0:
            notes[i,2] += time
            time = 0
        else:
            time += msg[2]                        #adjust time
    notes = notes[notes[:,1] > 0]                 #filter for notes with positive velocities (note presses)
    return notes

def NumpyNormalize(note, oneHot=False, full=False):                         #normalize all values to 0-1
    notes = np.copy(note)
    
    if oneHot:
        if full:
            notes[:,88] /= 128
            notes[:,89] /= 40000
            notes[:,90] /= 40000
        else:
            notes[:,12] /= 11
            notes[:,13] /= 128
            notes[:,14] /= 40000
            notes[:,15] /= 40000
    else:
        notes[:,0] /= 128
        notes[:,1] /= 128
        notes[:,2] /= 40000
        notes[:,3] /= 40000       
    return notes

def NumpyOneHot(note):
    notes = np.copy(note)
    oneHot = np.zeros([len(notes),16])
    oneHot[:, 13:] = notes[:, 1:]
    names = notes[:,0]
    namesOct = names%12
    oneHot[:,12] = (names-(namesOct))/12
    
    for i, name in enumerate(namesOct):
        oneHot[i,name.astype(int)] = 1
    
    return oneHot

def NumpyNotesOneHot(note):
    notes = np.copy(note)
    
    oneHot = np.zeros([len(notes),91])
    oneHot[:, 88:] = notes[:, 1:]
    names = notes[:,0]-21
    #namesOct = names%12
    #oneHot[:,12] = (names-(namesOct))/12
    
    for i, name in enumerate(names):
        oneHot[i,name.astype(int)] = 1
    
    return oneHot

def Midi2Numpy(path, oneHot=False, track0=1): # full midi to numpy conversion
    mid = MidiFile(path)
    notes = Midi2NumpyNoSustain(mid, track0=track0)
    cleanNotes = NumpyGetSustain(notes)
    
    if oneHot:
        cleanNotes = NumpyOneHot(cleanNotes)
    
    normNotes = NumpyNormalize(cleanNotes, oneHot=oneHot)
    return normNotes

#### Numpy to MIDI code

In [5]:
def NumpyDenormalize(note): # interpret all values from 0-1 to normal values
    notes = np.copy(note)    
    if notes.shape[1] == 16: # if encode as one-hot
        notes[:,12] *= 11 # octave
        notes[:,13] *= 128 # vel
        notes[:,14] *= 40000 # time
        notes[:,15] *= 40000 # sustain
        
        notes = NumpyEncode(notes) #encode back as original 4-variable format
        
    elif notes.shape[1] == 91: # if encoded as one-hot w/o octave
        notes[:,88] *= 128
        notes[:,89] *= 40000
        notes[:,90] *= 40000
        
        #print(notes)
        
        notes = NumpyEncodeNotes(notes)
        
    else:
        notes[:,0] *= 128
        notes[:,1] *= 128
        notes[:,2] *= 40000
        notes[:,3] *= 40000       
    return notes.astype(int)

def NumpyEncode(note): # convert back from one-hot encoding
    notes = np.copy(note)    
    encoded = np.zeros([len(notes),4]) # create array   
    encoded[:, 1:] = notes[:, 13:] # set vel/time/sustain
    encoded[:, 0] = notes[:,12]*12 # add octave value
    
    for i in range(len(notes)):
        encoded[i,0] += np.argmax(notes[i,:12])
    
    return encoded

def NumpyEncodeNotes(note): # convert back from one-hot encoding
    notes = np.copy(note)    
    encoded = np.zeros([len(notes),4])
    
    encoded[:, 1:] = notes[:, 88:] # set vel/time/sustain
    
    for i in range(len(notes)):
        encoded[i,0] += np.argmax(notes[i,:88])+21
    
    return encoded

def NumpySequence(notes): # put all notes into a "timeline" i.e.: time values of [10, 20, 10, 30] become [10, 30, 40, 70]
    sequenced = np.copy(notes)                      # this allows us to easily add vel=0 notes in any order since we can later sort them by time
    for i, msg in enumerate(sequenced):
        if i > 0:
            sequenced[i,2] += sequenced[i-1,2]
    return sequenced

def NumpyAddOffNotes(sequenced): # add vel=0 notes from sustain into sequenced timeline
    withOff = np.copy(sequenced)
    for msg in sequenced:
        offNote = np.array([[msg[0], 0, msg[2] + msg[3], 0]])
        withOff = np.append(withOff, offNote, axis=0)
    #withOff = np.sort(withOff,axis=0)
    withOff = withOff[withOff[:,2].argsort()] # sort by time
    return withOff

def NumpyUnsequence(notes): # revert time value to "time since last message"
    unsequenced = np.copy(notes)
    for i, msg in reversed(list(enumerate(unsequenced))):
        unsequenced[i,3] = 0
        if i > 0:
            unsequenced[i,2] -= unsequenced[i-1,2]
    return unsequenced

def Numpy2MidiDirect(array):    #make MIDI object from numpy
    #Start with initializing a new Mido Track:
    mid = MidiFile()
    track0 = MidiTrack()
    track1 = MidiTrack()
    
    track0.append(MetaMessage('set_tempo', tempo=500000, time=0)) #MetaMessages not necessary but are present in used files
    track0.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))
    track0.append(MetaMessage('end_of_track', time=1))
    
    track1.append(Message('program_change', channel=0, program=0, time=0))
    
    for i,note in enumerate(array):         # Get the index and the note. Array must be int array
        j = 1
        track1.append(Message('note_on',note = array[i,0], velocity = array[i,1],time = array[i,2])) # Add the note to the track.

    mid.tracks.append(track0)
    mid.tracks.append(track1)
    return mid

def Numpy2Midi(notes, name): # full numpy to midi conversion, saving result to [name].midi
    denorm = NumpyDenormalize(notes)
    seq = NumpySequence(denorm)
    off = NumpyAddOffNotes(seq)
    unseq = NumpyUnsequence(off)
    mid = Numpy2MidiDirect(unseq)
    mid.save(name + ".midi")

In [6]:
def splitTrack(midiFileName, inputDir, outputDir):
    mid = MidiFile(inputDir+midiFileName+".midi")
    for t, track in enumerate(mid.tracks):
        program, channel = findInstrument(track)        
        if len(track) < 200 or program == -1:
            continue
        trackMidi = copy.copy(mid)
        indices = [t]
        trackMidi.tracks = [trackMidi.tracks[x] for x in indices]
        #trackMidi.tracks.append(track)
        trackMidi.save(outputDir + midiFileName + "_prog{:0>3d}_chan{:0>2d}.midi".format(program,channel))

def splitAllTracks(inputDir, outputDir, first=0): # very bad code -  doesnt remove the .midi in the middle lol
    for i,f in enumerate(os.listdir(inputDir)):
        if i >= first:
            if len(f) <7:
                continue
            splitTrack(f[:7],inputDir,outputDir)

            if i % 1 == 0:
                print(f)
        
def findInstrument(track):
    count = 0
    infoMsg = 0
    
    for msg in track:
        if msg.type == "program_change":
            count += 1
            infoMsg = msg
    
    if count >= 1:
        if infoMsg.channel == 9 or count == 1:
            return infoMsg.program, infoMsg.channel
        else: 
            return -1, -1
    else:
        return -1, -1

In [7]:
#splitAllTracks("data/jazz/","data/jazz/tracks/")

#### Generatng tensor dataset from CSVs

In [8]:
def Numpy2Dataset(notes, tgt, num=100,skip=200): # make list of sumpy arrays
    samples = []
    targets = []
    i = 0
    while i+num <= len(notes):
        samples.append(notes[i:i+num])
        targets.append(tgt)
        i += skip
    return samples, targets

def SampleAllNumpy(dataPath): # generate samples from all saved CSVs
    allSamples = []
    allTargets = []

    for i,f in enumerate(os.listdir(dataPath)):
        notes = np.genfromtxt(dataPath+f, delimiter=',')
        tgt = GetGroup(f)
                             
        samples, targets = Numpy2Dataset(notes, tgt)
        allSamples += samples
        allTargets += targets
        if i % 100 == 0:
            print(i)
            
    random.seed(0)
    random.shuffle(allSamples)
    random.seed(0)
    random.shuffle(allTargets) # shuffle samples and targets in the exact same way
    
    return allSamples, allTargets

def SaveSamplesTensor(samples, targets, outputPath, name="Notes_Dataset"): # save tensor
    tens = torch.Tensor(samples)
    targ = torch.Tensor(targets)
    dataset = TensorDataset(tens,targ)
    torch.save(dataset, outputPath+name+".pt")
    return tens   

def SaveAllSamples(dataPath, outputPath, name="Notes_Dataset"): # save dataset tensor
    samples, targets = SampleAllNumpy(dataPath)
    SaveSamplesTensor(samples, targets, outputPath, name)
    
def SplitSamples(dataset, ranges):
    torch.maual_seed(0)
    trainData, valData, testData = random_split(oneHotDataset, splitRange)
    return trainData, valData, testData 

In [9]:
def ProgramGroup(program, channel):
    if channel == 9:
        return 0 # drums
    if program < 8:
        return 1 # piano
    if program < 16:
        return 8 # pitched percussion
    if program < 24:
        return 7 # organ
    if program < 32:
        return 3 # guitar
    if program < 40:
        return 2 # bass
    if program < 48:
        return 9 # string
    if program < 56:
        return 10 # ensemble
    if program < 64:
        return 4 # brass 
    if program < 72:
        return 5 # reed
    if program < 80:
        return 11 # pipe
    if program < 88:
        return 6 # synth lead
    if program < 96:
        return 12 # synth pad
    if program < 104:
        return 13 # synth efects
    if program < 112:
        return 14 # ethnic
    if program < 120:
        return 15 # percusson
    else:
        return 16 # other 
    
def GetProgram(name):
    prog = name[12:15]
    chan = name[20:22]
    return int(prog), int(chan)

def GetGroup(name):
    prog, chan = GetProgram(name)
    group = ProgramGroup(prog, chan)
    return(group)

#### Bulk data conversion code - COMMENT OUT IF NOT IN USE!!!

In [10]:
#SaveAllSamples("data/jazz/tracks_numpy/train/","data/","trainDataClass") #save all into tensor
#SaveAllSamples("data/jazz/tracks_numpy/val/","data/","valDataClass") #save all into tensor
#SaveAllSamples("data/jazz/tracks_numpy/test/","data/","testDataClass") #save all into tensor

# oneHotDataset = torch.load("data/onehot_data/onehot.pt")

# splitProp = np.array([0.9, 0.05 , 0.05]) # 90/5/5% split
# splitRange = len(oneHotDataset)*splitProp
# splitRange[0] += 1                           # so numbers add up
# splitRange = splitRange.astype(int)

# length = len(oneHotDataset)
# a = 0.9*length
# a = int(a)

# b = 0.95*length
# b = int(b)

# tiny = oneHotDataset[:30]
# train = oneHotDataset[:a]
# val = oneHotDataset[a:b]
# test = oneHotDataset[b:]

# tinyData = TensorDataset(tiny[0].clone(), tiny[1].clone())
# trainData = TensorDataset(train[0].clone(),train[1].clone())
# valData = TensorDataset(val[0].clone(),val[1].clone())
# testData = TensorDataset(test[0].clone(),test[1].clone())

# torch.save(tinyData,"data/onehot_data/tinyDataOneHot.pt")
# torch.save(trainData,"data/onehot_data/trainDataOneHot.pt")
# torch.save(valData,"data/onehot_data/valDataOneHot.pt")
# torch.save(testData,"data/onehot_data/testDataOneHot.pt")

In [11]:
#a = torch.load("data/va.pt")

In [12]:
# for img, label in iter(a):
#     #print(img)
#     print(label)
#     print()

In [13]:
# IMPORTANT: COMMENT OUT IF NOT IN USE TO AVOID ACCIDENTS!!!!!!!

# Getting CSVs from MIDI data and processed data from CSVs
# Processed MIDI does not contain program messages and so are a good measure of what output SHOULD look like in a perfect world

# dataPath = "data/jazz/tracks/"
# outputPath = "data/jazz/tracks_numpy/"
# #processedPath = "data/MIDI_files_processed/"

# for i,f in enumerate(os.listdir(dataPath)):
#     if i > 2700:
#         notes = Midi2Numpy(dataPath+f,track0=0)
#         np.savetxt(outputPath + f[:22]+".csv",notes,delimiter=",")
#     #Numpy2Midi(notes, processedPath + "MIDI_{:04d}".format(i))
#     #break
    
#     if i % 200 == 0:
#         print(i)

In [14]:
# IMPORTANT: COMMENT OUT IF NOT IN USE TO AVOID ACCIDENTS!!!!!!!

# Getting CSVs from MIDI data and processed data from CSVs
# Processed MIDI does not contain program messages and so are a good measure of what output SHOULD look like in a perfect world

# dataPath = "data/jazz/tracks/"
# outputPath = "data/jazz/tracks_numpy/"
# #processedPath = "data/MIDI_files_processed/"

# for i,f in enumerate(os.listdir(dataPath)):
#     if i == 47:
#         print(f)
#         testMidi = MidiFile(dataPath+f)
#         #PrintMessages(testMidi)
#         print(GetGroup(f))
#     #Numpy2Midi(notes, processedPath + "MIDI_{:04d}".format(i))
#     #break
    
#     if i % 100 == 0:
#         print(i)

In [15]:
# dataPath = "data/numpy_files/"  # one-hot encoding on CSVs
# outputPath = "data/numpy_full/"

# for i,f in enumerate(os.listdir(dataPath)):
#     notes = np.genfromtxt(dataPath+f, delimiter=',')
#     notes = NumpyDenormalize(notes)
#     notes = NumpyNotesOneHot(notes)
#     notes = NumpyNormalize(notes, oneHot=True, full=True)
#     np.savetxt(outputPath + "MIDI_{:04d}.csv".format(i),notes,delimiter=",")
#     if i % 100 == 0:
#         print(i)

## Baseline Model Code
#### getting available notes

## Model architecture definition

Set the hyperparameters below:


In [16]:
torch.manual_seed(1)

class LSTMClassifier(nn.Module):
    def __init__(self, n_features, n_classes, n_hidden, n_layers):
        super(LSTMClassifier, self).__init__()
        self.name = "LSTMClassifier"
        self.n_features, self.n_classes, self.n_hidden, self.n_layers = n_features, n_classes, n_hidden, n_layers, 
        self.lstm = nn.LSTM(n_features, n_hidden, n_layers, batch_first=True,dropout = 0.3)
        self.fc = torch.nn.Linear(2*n_hidden, n_classes)

    def forward(self, x):
        # Forward propagate the RNN
        h0 = torch.zeros(self.n_layers,x.size(0),self.n_hidden)
        c0 = torch.zeros(self.n_layers,x.size(0),self.n_hidden)
        if use_cuda and torch.cuda.is_available():
            c0 = c0.cuda()
            h0 = h0.cuda()
        out, _ = self.lstm(x)
        # Pass the output of the last time step to the classifier
        out = torch.cat([torch.max(out, dim=1)[0], 
                    torch.mean(out, dim=1)], dim=1) #Combine max and mean
        output = self.fc(out)
        return output

print('Model class created succesfully')

Model class created succesfully


## Training function

In [17]:
#To help us save the model easier...
def get_model_name(name, batch_size, learning_rate, epoch):
    """ Generate a name for the model consisting of all the hyperparameter values

    Args:
        config: Configuration object containing the hyperparameters
    Returns:
        path: A string with the hyperparameter name and value concatenated
    """
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
                                                   batch_size,
                                                   learning_rate,
                                                   epoch)
    return path

In [18]:
def get_accuracy(model, data_loader): #Accuracy on note selection...
    correct = 0
    total = 0
    i = 0
    model.eval()
    for sample in data_loader:
        inputs, labels = sample
        if use_cuda and torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()
        output = model(inputs)
        pred = output.max(1, keepdim=True)[1]
        if i%100 == 0:
            print(pred)
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += len(labels)
        i += 1
    return (correct / total)*100



In [19]:
# Training Curve
def plot_training_curve(accLoss, train, val):
    """ Plots the training curve for a model's train/validation loss or accuracy.

    Args:
        accLoss: "Accuracy" or "Loss"
        train: train data
        val: validation data
    """
    import matplotlib.pyplot as plt
    titleStr = "Train vs Validation " + accLoss
    plt.title(titleStr)
    n = len(train) # number of epochs
    plt.plot(range(1,n+1), train, label="Train")
    if val != False:
        
        plt.plot(range(1,n+1), val, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel(accLoss)
    plt.legend(loc='best')
    plt.show()

In [20]:
def train(model, train_dataset, val_dataset, num_epochs=5, batch_size=64, learning_rate=1e-3,model_name = 'model'):
    torch.manual_seed(1000) #Fixed. Make sure we use this throughout...
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=learning_rate, weight_decay=0.001)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    
    start_time=time.time() #Start of training

    train_losses, train_accs, val_accs = [], [], []

    for epoch in range(num_epochs):
        for i, data in enumerate(train_loader):
            inputs, labels = data
            if use_cuda and torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()
#             print('labels: ',labels)
            model.train()
            
            out = model(inputs)             # forward pass
#             print(out.shape)
            loss = criterion(out, labels) # compute the total loss

            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch

            loss = float(loss)/batch_size             # compute *average* loss

        train_acc = get_accuracy(model, train_loader)
#         val_loss = get_loss(model, val_loader, criterion)
        val_acc =get_accuracy(model, val_loader)

        print('Epoch: {} - Train loss: {:.4f}, Train accuracy: {:.2f}%, Validation accuracy: {:.2f}%'.format(
                            epoch+1, float(loss), float(train_acc), float(val_acc)))
        
        train_losses.append(loss)             # compute *average* loss
        train_accs.append(train_acc)
#         val_losses.append(val_loss)
        val_accs.append(val_acc)

    #Save the file current model with the weights
    model_path = get_model_name(model.name, batch_size, learning_rate, num_epochs)
    torch.save(model.state_dict(), model_path)
    end_time= time.time()
    torch.save(model.state_dict(), model_name+".pt")
    return train_losses, train_accs, val_accs

In [21]:
def get_class_binary(dataset,label_1,label_2):
    list_data = list(dataset)
    data = []
    label = []

    for sample in list_data:
        data.append((sample[0]).detach().numpy())
        label.append((sample[1]).detach().numpy())

    data = np.array(data)
    label = np.array(label)

    # Grab data that are classified as 0 or 1
    class_0_1_data = data[np.where(np.logical_or(label == label_1, label == label_2))]
    class_0_1_label = label[np.where(np.logical_or(label == label_1, label == label_2))]

    # Prevent type errors
    out_dataset = TensorDataset(torch.tensor(class_0_1_data).float(), torch.tensor(class_0_1_label).long())
    return out_dataset

In [22]:
def get_class_3(dataset,label_1,label_2,label_3): #For any 3 classes.
    list_data = list(dataset)
    data = []
    label = []

    for sample in list_data:
        data.append((sample[0]).detach().numpy())
        label.append((sample[1]).detach().numpy())
    
    data = np.array(data)
    label = np.array(label)
#     print(label)
    # Grab data that are classified as 0 or 1
    class_0_1_data = data[np.where(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)))]
    class_0_1_label = label[np.where(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)))]
    print(class_0_1_label)
    # Prevent type errors
    out_dataset = TensorDataset(torch.tensor(class_0_1_data).float(), torch.tensor(class_0_1_label).long())
    return out_dataset

In [23]:
def get_class_9(dataset,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9): #For any 9 classes.
    list_data = list(dataset)
    data = []
    label = []

    for sample in list_data:
        data.append((sample[0]).detach().numpy())
        label.append((sample[1]).detach().numpy())
    
    data = np.array(data)
    label = np.array(label)
#     print(label)
    # Grab data that are classified as 0 or 1
    class_0_1_data = data[np.where(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)),label==label_4),label==label_5),label==label_6),label==label_7),label==label_8),label==label_9))]
    class_0_1_label = label[np.where(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)),label==label_4),label==label_5),label==label_6),label==label_7),label==label_8),label==label_9))]
    print(class_0_1_label)
    # Prevent type errors
    out_dataset = TensorDataset(torch.tensor(class_0_1_data).float(), torch.tensor(class_0_1_label).long())
    return out_dataset

In [24]:
def get_class_8(dataset,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8): #For any 8 classes.
    list_data = list(dataset)
    data = []
    label = []

    for sample in list_data:
        data.append((sample[0]).detach().numpy())
        label.append((sample[1]).detach().numpy())
    
    data = np.array(data)
    label = np.array(label)
#     print(label)
    # Grab data that are classified as 0 or 1
    class_0_1_data = data[np.where(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)),label==label_4),label==label_5),label==label_6),label==label_7),label==label_8))]
    class_0_1_label = label[np.where(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)),label==label_4),label==label_5),label==label_6),label==label_7),label==label_8))]
    print(class_0_1_label)
    # Prevent type errors
    out_dataset = TensorDataset(torch.tensor(class_0_1_data).float(), torch.tensor(class_0_1_label).long())
    return out_dataset

In [25]:
def get_class_7(dataset,label_1,label_2,label_3,label_4,label_5,label_6,label_7): #For any 7 classes.
    list_data = list(dataset)
    data = []
    label = []

    for sample in list_data:
        data.append((sample[0]).detach().numpy())
        label.append((sample[1]).detach().numpy())
    
    data = np.array(data)
    label = np.array(label)
#     print(label)
    # Grab data that are classified as 0 or 1
    class_0_1_data = data[np.where(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)),label==label_4),label==label_5),label==label_6),label==label_7))]
    class_0_1_label = label[np.where(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)),label==label_4),label==label_5),label==label_6),label==label_7))]
    print(class_0_1_label)
    # Prevent type errors
    out_dataset = TensorDataset(torch.tensor(class_0_1_data).float(), torch.tensor(class_0_1_label).long())
    return out_dataset

In [26]:
def get_class_6(dataset,label_1,label_2,label_3,label_4,label_5,label_6): #For any 6 classes.
    list_data = list(dataset)
    data = []
    label = []

    for sample in list_data:
        data.append((sample[0]).detach().numpy())
        label.append((sample[1]).detach().numpy())
    
    data = np.array(data)
    label = np.array(label)
#     print(label)
    # Grab data that are classified as 0 or 1
    class_0_1_data = data[np.where(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)),label==label_4),label==label_5),label==label_6))]
    class_0_1_label = label[np.where(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)),label==label_4),label==label_5),label==label_6))]
    print(class_0_1_label)
    # Prevent type errors
    out_dataset = TensorDataset(torch.tensor(class_0_1_data).float(), torch.tensor(class_0_1_label).long())
    return out_dataset

In [27]:
def get_class_5(dataset,label_1,label_2,label_3,label_4,label_5): #For any 5 classes.
    list_data = list(dataset)
    data = []
    label = []

    for sample in list_data:
        data.append((sample[0]).detach().numpy())
        label.append((sample[1]).detach().numpy())
    
    data = np.array(data)
    label = np.array(label)
#     print(label)
    # Grab data that are classified as 0 or 1
    class_0_1_data = data[np.where(np.logical_or(np.logical_or(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)),label==label_4),label==label_5))]
    class_0_1_label = label[np.where(np.logical_or(np.logical_or(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)),label==label_4),label==label_5))]
    print(class_0_1_label)
    # Prevent type errors
    out_dataset = TensorDataset(torch.tensor(class_0_1_data).float(), torch.tensor(class_0_1_label).long())
    return out_dataset

In [28]:
def get_class_4(dataset,label_1,label_2,label_3,label_4): #For any 4 classes.
    list_data = list(dataset)
    data = []
    label = []

    for sample in list_data:
        data.append((sample[0]).detach().numpy())
        label.append((sample[1]).detach().numpy())
    
    data = np.array(data)
    label = np.array(label)
#     print(label)
    # Grab data that are classified as 0 or 1
    class_0_1_data = data[np.where(np.logical_or(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)),label==label_4))]
    class_0_1_label = label[np.where(np.logical_or(np.logical_or(np.logical_or((label == label_1), (label == label_2)), (label == label_3)),label==label_4))]
    print(class_0_1_label)
    # Prevent type errors
    out_dataset = TensorDataset(torch.tensor(class_0_1_data).float(), torch.tensor(class_0_1_label).long())
    return out_dataset

In [29]:
def get_class(dataset):
    list_data = list(dataset)
    data = []
    label = []

    for sample in list_data:
        data.append((sample[0]).detach().numpy())
        label.append((sample[1]).detach().numpy())
    
    data = np.array(data)
    label = np.array(label)
#     print(label)
    # Grab data that are classified as 0 or 1,2,3,4,5,6,7,8,9... Other classes have 0 samples.
    class_0_1_data = data[np.where(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or((label == 0), (label == 1)), (label == 2)),label==4),label==5),label==6),label==7),label==8),label==9),label==3))]
    class_0_1_label = label[np.where(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or(np.logical_or((label == 0), (label == 1)), (label == 2)),label==4),label==5),label==6),label==7),label==8),label==9),label==3))]
    print(class_0_1_label)
    # Prevent type errors
    out_dataset = TensorDataset(torch.tensor(class_0_1_data).float(), torch.tensor(class_0_1_label).long())
    return out_dataset

In [30]:
# Load saved tensor dataset
'''
For colab:
'''
# train_dataset = torch.load(r'/content/gdrive/My Drive/APS360/Project/trainDataClass.pt')
# val_dataset = torch.load(r'/content/gdrive/My Drive/APS360/Project/valDataClass.pt')
# test_dataset = torch.load(r'/content/gdrive/My Drive/APS360/Project/testDataClass.pt')
'''
For local: (@Kevin)
'''
'''
Normal:
'''
# train_dataset = torch.load(r'D:\engsci\year 3\CLASS\APS360\project_pivot\dataset\ClassData (1)\normal\trainDataClass.pt')
# val_dataset = torch.load(r'D:\engsci\year 3\CLASS\APS360\project_pivot\dataset\ClassData (1)\normal\valDataClass.pt')
# test_dataset = torch.load(r'D:\engsci\year 3\CLASS\APS360\project_pivot\dataset\ClassData (1)\normal\testDataClass.pt')

'''
One hot:
'''
train_dataset = torch.load(r'D:\engsci\year 3\CLASS\APS360\project_pivot\dataset\ClassData (1)\onehot\trainDataClassOnehot.pt')
val_dataset = torch.load(r'D:\engsci\year 3\CLASS\APS360\project_pivot\dataset\ClassData (1)\onehot\valDataClassOnehot.pt')
test_dataset = torch.load(r'D:\engsci\year 3\CLASS\APS360\project_pivot\dataset\ClassData (1)\onehot\testDataClassOnehot.pt')

In [31]:
# # Print stats from the dataset
# for imgs, labels in torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True):
# #     print(labels)
#     labels = labels

# hist_var = np.zeros((8))
# for i in range(len(labels)):
# #     print(labels[i].numpy())
#     #Count each class samples...
#     for j in range(0,8):
#         if labels[i] == j:
#             hist_var[j] += 1
# print(hist_var) #Good visual for the demo! (see plot below :) )


# plt.plot(np.linspace(0,7,8),hist_var,"o")
# plt.ylabel('Number of Samples')
# plt.xlabel('Class')
# plt.title('TrainData Class Distributions')
# plt.show()

# for imgs, labels in torch.utils.data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True):
# #     print(labels)
#     labels = labels

# hist_var = np.zeros((8))
# for i in range(len(labels)):
# #     print(labels[i].numpy())
#     #Count each class samples...
#     for j in range(0,8):
#         if labels[i] == j:
#             hist_var[j] += 1
# print(hist_var) #Good visual for the demo! (see plot below :) )


# plt.plot(np.linspace(0,7,8),hist_var,"o")
# plt.ylabel('Number of Samples')
# plt.xlabel('Class')
# plt.title('valData Class Distributions')
# plt.show()

# for imgs, labels in torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True):
# #     print(labels)
#     labels = labels

# hist_var = np.zeros((8))
# for i in range(len(labels)):
# #     print(labels[i].numpy())
#     #Count each class samples...
#     for j in range(0,8):
#         if labels[i] == j:
#             hist_var[j] += 1
# print(hist_var) #Good visual for the demo! (see plot below :) )


# plt.plot(np.linspace(0,7,8),hist_var,"o")
# plt.ylabel('Number of Samples')
# plt.xlabel('Class')
# plt.title('testData Class Distributions')
# plt.show()

In [32]:
# train_dataset = get_class_3(train_dataset,0,1,2)
# val_dataset = get_class_3(val_dataset,0,1,2)
# test_dataset = get_class_3(test_dataset,0,1,2)

#Run the 2,5,6
train_dataset = get_class_7(train_dataset,0,1,2,3,4,5,6)
val_dataset = get_class_7(val_dataset,0,1,2,3,4,5,6)
test_dataset = get_class_7(test_dataset,0,1,2,3,4,5,6)

[4. 0. 5. ... 5. 4. 3.]
[6. 2. 4. ... 0. 2. 3.]
[1. 0. 1. ... 0. 3. 0.]


In [33]:
use_cuda = True

# For LSTM: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html #torch.nn.LSTM 
# Hyperparameters
# 8 features overfits. Trying with dropout.
N_FEATURES = 131
N_CLASSES = 7
N_HIDDEN = 32
N_LAYERS = 3
BATCH_SIZE = 128
NUM_EPOCHS = 40
LEARNING_RATE = 1e-3
model = LSTMClassifier(n_features=N_FEATURES, n_classes = N_CLASSES, n_hidden=N_HIDDEN, n_layers=N_LAYERS)
if use_cuda and torch.cuda.is_available():
    model.cuda()

In [ ]:
train_losses, train_accs, val_accs = train(model, train_dataset, val_dataset, num_epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, learning_rate=LEARNING_RATE,model_name='8_class_model_onehot')

tensor([[1],
        [4],
        [4],
        [2],
        [1],
        [4],
        [2],
        [1],
        [6],
        [2],
        [1],
        [0],
        [1],
        [2],
        [1],
        [2],
        [2],
        [0],
        [1],
        [1],
        [2],
        [1],
        [2],
        [2],
        [4],
        [2],
        [3],
        [2],
        [2],
        [2],
        [3],
        [2],
        [2],
        [0],
        [2],
        [1],
        [2],
        [2],
        [1],
        [4],
        [4],
        [2],
        [6],
        [1],
        [6],
        [2],
        [2],
        [1],
        [6],
        [0],
        [1],
        [6],
        [2],
        [6],
        [2],
        [4],
        [1],
        [6],
        [1],
        [4],
        [2],
        [6],
        [2],
        [0],
        [2],
        [1],
        [3],
        [4],
        [6],
        [2],
        [4],
        [2],
        [2],
        [4],
        [2],
        [6],
        [6],

In [ ]:
# Plot loss curves
plot_training_curve("Loss", train_losses,False)

# Plot accuracy curves
plot_training_curve("Accuracy", train_accs, val_accs)

In [ ]:
from sklearn.metrics import confusion_matrix
print(len(train_dataset))
print(len(val_dataset))
model = model.cpu()
model.eval()

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True)
for i, data in enumerate(train_loader):
    inputs, y_true = data

#             print('labels: ',labels)
    output = model(inputs)             # forward pass
    y_pred = output.max(1, keepdim=True)[1]
print(y_pred.shape)
y_pred = y_pred.detach().numpy()
y_true = y_true.detach().numpy()
# print(y_pred)
print("confusion matrix from trainin set") #https://codeyarns.com/tech/2014-10-24-how-to-create-a-confusion-matrix-plot-using-matplotlib.html
m = confusion_matrix(y_true,y_pred,normalize = 'true')
plt.matshow(m)
plt.colorbar()

for i, data in enumerate(val_loader):
    inputs, y_true = data

#             print('labels: ',labels)
    output = model(inputs)             # forward pass
    y_pred = output.max(1, keepdim=True)[1]
print(y_pred.shape)
y_pred = y_pred.detach().numpy()
y_true = y_true.detach().numpy()
# print(y_pred)
print("confusion matrix from validation set") #https://codeyarns.com/tech/2014-10-24-how-to-create-a-confusion-matrix-plot-using-matplotlib.html
m = confusion_matrix(y_true,y_pred,normalize = 'true')
plt.matshow(m)
plt.colorbar()


## Model Testing

In [ ]:
#Only run at the end...
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
# model = model.cuda()
# criterion = nn.CrossEntropyLoss()

# test_acc = get_accuracy(model, test_loader)
# test_loss = get_loss(model, test_loader, criterion)
# model = model.cpu()
# model.eval()
# print(("Test Loss: {:.4f}").format(float(test_loss)))
# print(("Test Accuracy: {:.2f}%").format(float(test_acc)))
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True)
# for i, data in enumerate(test_loader):
#     inputs, y_true = data

# #             print('labels: ',labels)
#     output = model(inputs)             # forward pass
#     y_pred = output.max(1, keepdim=True)[1]
# print(y_pred.shape)
# y_pred = y_pred.detach().numpy()
# y_true = y_true.detach().numpy()
# # print(y_pred)
# print("confusion matrix from validation set") #https://codeyarns.com/tech/2014-10-24-how-to-create-a-confusion-matrix-plot-using-matplotlib.html
# m = confusion_matrix(y_true,y_pred,normalize = 'true')
# plt.matshow(m)
# plt.colorbar()

In [ ]:
# model = LSTMClassifier(n_features=N_FEATURES, n_classes = N_CLASSES, n_hidden=N_HIDDEN, n_layers=N_LAYERS)

# model.load_state_dict(torch.load('D:/engsci/year 3/CLASS/APS360/project_pivot/7_class_model.pt'))